In [ ]:
!pip install pymorphy2 selenium pymupdf
!apt-get update
!apt install chromium-chromedriver
!pip install lightgbm xgboost

In [ ]:
from ast import literal_eval
import os, json, time, hashlib, zipfile, re
import warnings, gc, sys
import requests, fitz
from itertools import islice
from io import BytesIO
import urllib3, multiprocessing
from urllib.request import urlopen
from urllib.parse import urlparse
from selenium import webdriver
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup
from IPython.display import display
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (ExtraTreesClassifier, GradientBoostingClassifier,
                              RandomForestClassifier)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score,
                             classification_report, confusion_matrix, f1_score,
                             make_scorer, precision_score, recall_score,
                             roc_auc_score)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
import xgboost as xgb

nltk.download('stopwords')
nltk.download('punkt')
urllib3.disable_warnings()
warnings.filterwarnings("ignore")
np.set_printoptions(threshold=sys.maxsize)

%matplotlib inline

# Functions

In [ ]:
def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        original_filename = os.path.basename(url)
        truncated_filename = truncate_filename(original_filename)
        file_path = os.path.join('/content/PDFs/', truncated_filename)
        with open(file_path, 'wb') as file:
            file.write(response.content)
        return file_path
    else:
        print(f"Failed to download PDF: {url, response.status_code}")
        return None

In [ ]:
def truncate_filename(filename, max_length=255):
    if len(filename) > max_length:
        hash_part = hashlib.md5(filename.encode()).hexdigest()
        extension = os.path.splitext(filename)[1]
        filename = filename[:max_length - len(hash_part) - len(extension) - 1] + '-' + hash_part + extension
    return filename

In [ ]:
def clear_string(url):
    parts = url.split(' ')
    result = []
    i = 0
    if i + 2 < len(parts) and parts[i+1] == 'и':
        result.append(parts[i])
        result.append(parts[i+2])
    else:
        result.append(parts[i])
    return result

In [ ]:
def get_all_links(company, url):
    if ".pdf" in url.lower():
        internalLinks = []
        if "http" in url:
            filename = download_pdf(url)
            if filename is not None:
                internalLinks.append(filename)

                data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
                data['company'] = company
                data = data[['company', 'urls']]

                return data

        else:
            if "/content/PDFs/" not in url:
                url = os.path.join('/content/PDFs/', url)

            internalLinks.append(url)

            data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
            data['company'] = company
            data = data[['company', 'urls']]

            return data
    else:
        internalLinks = [url]
        status = 0
        try:
            response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
            status = response.status_code
        except Exception as err:
            pass

        if status == 200:
            soup = BeautifulSoup(response.content, 'lxml')
        else:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--disable-extensions')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            driver = webdriver.Chrome(options=chrome_options)

            driver.set_page_load_timeout(10)

            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.quit()
            del driver
            gc.collect()

        for script in soup(["script", "style"]):
            script.extract()

        sublinks = [a.get('href') for a in soup.find_all('a') if a.get('href') and a.get('href').startswith('/')]
        sublinks = [link[1:] for link in sublinks]
        sublinks = [*set(sublinks)]
        sublinks = list(filter(None, sublinks))

        for link in sublinks:
            if ".pdf" not in link.lower():
                if 'www' in link:
                    new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
                else:
                    new_url = os.path.join(url[:url.rfind('//')+2], urlparse(url).netloc, link)
                internalLinks.append(new_url)

        data = pd.DataFrame(internalLinks, columns=['urls']).drop_duplicates()
        data['company'] = company
        data = data[['company', 'urls']]

        return data

In [ ]:
def parsing(company, url):
    if ".pdf" in url.lower():
        if "http" in url:
            filename = download_pdf(url)
            if filename is not None:
                reader = fitz.open(f"{filename}")
                text = ""
                for page in range(reader.page_count):
                    text += reader.load_page(page).get_text() + "\n"

                text = text.replace('-\n', '').lower()
                text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
                text = ''.join([i for i in text if not i.isdigit()])

                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                text = '\n'.join(chunk for chunk in chunks if chunk)

                text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

                text_lemmatized = []
                for line in [t.split() for t in text]:
                    line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
                    text_lemmatized.append(line_lemmatized)

                data = pd.DataFrame(text_lemmatized, columns=['text']).drop_duplicates()
                data['company'] = company
                data['url'] = url
                data = data[['company', 'url', 'text']]

                return data
        else:
            if "/content/PDFs/" not in url:
                url = os.path.join('/content/PDFs/', url)

            reader = fitz.open(f"{url}")
            text = ""
            for page in range(reader.page_count):
                text += reader.load_page(page).get_text() + "\n"

            text = text.replace('-\n', '').lower()
            text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
            text = ''.join([i for i in text if not i.isdigit()])

            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)

            text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

            text_lemmatized = []
            for line in [t.split() for t in text]:
                line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
                text_lemmatized.append(line_lemmatized)

            data = pd.DataFrame(text_lemmatized, columns=['text']).drop_duplicates()
            data['company'] = company
            data['url'] = url
            data = data[['company', 'url', 'text']]

            return data

    else:
        status = 0
        try:
            response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
            status = response.status_code
        except Exception as err:
            pass

        if status == 200:
            soup = BeautifulSoup(response.content, 'lxml')
        else:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--no-sandbox')
            chrome_options.add_argument('--disable-gpu')
            chrome_options.add_argument('--disable-extensions')
            chrome_options.add_argument('--disable-dev-shm-usage')
            chrome_options.add_argument('--disable-blink-features=AutomationControlled')
            driver = webdriver.Chrome(options=chrome_options)

            driver.set_page_load_timeout(10)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            driver.quit()
            del driver
            gc.collect()

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk).replace('-\n', '').lower()
        text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
        text = ''.join([i for i in text if not i.isdigit()])

        text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

        text_lemmatized = []
        for line in [t.split() for t in text]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            text_lemmatized.append(line_lemmatized)

        spans = list(filter(None, [span.string for span in soup.find_all('span')]))

        if len(spans):
            spans = '\n'.join(span for span in spans if span).replace('-\n', '').lower()
            spans = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', spans)
            spans = ''.join([i for i in spans if not i.isdigit()])

            spans = list(np.concatenate([sent_tokenize(i.strip()) for i in spans.split('\n')]).flat)
        else:
            spans = []

        spans_lemmatized = []
        for line in [s.split() for s in spans]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            spans_lemmatized.append(line_lemmatized)

        df1 = pd.DataFrame(text_lemmatized, columns=['text'])
        df2 = pd.DataFrame(spans_lemmatized, columns=['text'])
        data = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
        data['company'] = company
        data['url'] = url
        data = data[['company', 'url', 'text']]

        return data

In [ ]:
def chunkify(data, chunk_size):
    it = iter(data)
    for i in range(0, len(data), chunk_size):
        yield {k: data[k] for k in [next(it) for _ in range(min(chunk_size, len(data) - i))]}

def process_chunk(URLs_chunk, enum, function):
    with ThreadPoolExecutor(4) as executor:
        chunk_res = pd.DataFrame()
        results = {executor.submit(function, company, url): (url, company) for url, company in URLs_chunk.items()}
        for i, result in enumerate(as_completed(results)):
            try:
                url, company = results[result]
                df1 = result.result()
                chunk_res = pd.concat([chunk_res, df1])
                print('DONE', i+enum, company)
            except Exception as err:
                print('ERROR', i+enum, company, err)
                continue
    return chunk_res, i+enum+1

In [ ]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w') as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

In [ ]:
def rating_band_prediction(features, target, best_models = None, features_data_test = None):
    if target != 'ESG_Rating_band_by_E/S/G_Rating_band':
        X = features_data.drop(columns=['ESG_Rank', 'ESG_Rating_band', 'E_Rank', 'E_Rating_band', 'S_Rank', 'S_Rating_band', 'G_Rank', 'G_Rating_band', 'company'])
        y = features_data[target]
        #display(y.value_counts())
        label_encoder = LabelEncoder().fit(y)

    param_grid_lr = {'classifier__C': [0.1, 1, 10], 'classifier__penalty': ['l1', 'l2'], 'classifier__solver': ['lbfgs'],
                      'classifier__multi_class': ['ovr'], 'classifier__max_iter': [10, 100, 200, 500]}
    param_grid_rf = {'classifier__n_estimators': [50, 100, 200], 'classifier__criterion': ['gini', 'entropy', 'log_loss'],
                      'classifier__max_depth': [None, 1, 2, 3, 5], 'classifier__min_samples_split': [2, 5, 10]}
    param_grid_svc = {'classifier__C': [0.1, 0.2, 0.5, 1, 10], 'classifier__kernel': ['linear', 'rbf', 'poly'],
                      'classifier__gamma': ['scale', 'auto']}
    param_grid_nb = {'classifier__var_smoothing': [1e-9, 1e-8, 1e-7]}
    param_grid_dt = {'classifier__criterion': ['gini', 'entropy', 'classifier__log_loss'], 'classifier__max_depth': [None, 1, 2, 3 , 5, 10, 50, 100, 200]}
    param_grid_knn = {'classifier__n_neighbors': [2, 3, 5, 7, 10], 'classifier__weights': ['uniform', 'distance'], 'classifier__metric': ['euclidean', 'manhattan']}
    param_grid_gbm = {'classifier__n_estimators': [10, 50, 100, 200], 'classifier__learning_rate': [0.001, 0.01, 0.1, 0.2], 'classifier__max_depth': [None, 1, 2, 3, 4, 5]}
    param_grid_xgb = {'classifier__objective': ['multi:softprob'], 'classifier__n_estimators': [50, 100, 200], 'classifier__learning_rate': [0.001, 0.01, 0.1],
                      'classifier__max_depth': [None, 1, 2, 3, 4, 5], 'classifier__subsample': [0.8, 1.0], 'classifier__colsample_bytree': [0.8, 1.0]}
    param_grid_lgb = {'classifier__objective': ['multiclass'], 'classifier__n_estimators': [50, 100, 200], 'classifier__learning_rate': [0.001, 0.01, 0.1],
                      'classifier__num_leaves': [1, 5, 10, 30, 50, 100], 'classifier__boosting_type': ['gbdt', 'dart'], 'classifier__max_depth': [None, 1, 2, 3, 4, 5]}

        # models = {'Logistic Regression': (LogisticRegression(), param_grid_lr),
        #     'Decition Tree': (DecisionTreeClassifier(random_state=42), param_grid_dt),
        #     'Random Forest': (RandomForestClassifier(random_state=42), param_grid_rf),
        #     'Gradient Boosting': (GradientBoostingClassifier(random_state=42), param_grid_gbm),
        #     'SVM': (SVC(probability=True, random_state=42), param_grid_svc),
        #     'Naive Bayes': (GaussianNB(), param_grid_nb),
        #     'KNN': (KNeighborsClassifier(), param_grid_knn),
        #     'LightGBM': (lgb.LGBMClassifier(random_state=42), param_grid_lgb),
        #     'XGBoost': (xgb.XGBClassifier(use_label_encoder=False, eval_metric='f1_score', random_state=42), param_grid_xgb)}

        # param_grids = [{}, {'classifier__n_neighbors': [2, 5, 10, 20, 50, 100], 'classifier__weights': ['uniform', 'distance']},
        #            {'classifier__base_estimator__penalty': ['l1', 'l2'], 'classifier__base_estimator__C': [0.1, 0.2, 0.5, 1],
        #            'classifier__base_estimator__max_iter': [100, 200, 500, 1000, 5000, 10000], 'classifier__base_estimator__multi_class': ['ovr'],
        #            'classifier__base_estimator__random_state': [42]}, {'classifier__penalty': ['l1', 'l2'], 'classifier__Cs': [1, 10],
        #            'classifier__max_iter': [10, 100, 200, 500], 'classifier__solver' : ['lbfgs'], 'classifier__multi_class': ['ovr'],
        #            'classifier__random_state': [42]}, {'classifier__criterion': ['gini', 'entropy', 'log_loss'], 'classifier__max_depth': [None, 1, 2, 3 , 5, 10, 50, 100, 200],
        #            'classifier__random_state': [42]}, {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ['gini'], 'classifier__random_state': [42]},
        #            {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ['gini', 'entropy', 'log_loss'], 'classifier__random_state': [42], 'classifier__max_depth': [None, 3, 10, 20]},
        #            {'classifier__n_estimators': [10, 50, 100, 200, 500], 'classifier__max_depth': [None, 1, 2, 3, 10, 20], 'classifier__random_state': [42]}]

    model_names = ['LogisticRegression', 'GaussianNaiveBayes', 'SupportVectorMachine', 'KNeighborsClassifier',
                  'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier', 'LightGBM', 'XGBoost']
    names_in_python = [LogisticRegression(random_state=42), GaussianNB(), SVC(probability=True, random_state=42), KNeighborsClassifier(),
                      DecisionTreeClassifier(random_state=42), RandomForestClassifier(random_state=42), GradientBoostingClassifier(random_state=42),
                        lgb.LGBMClassifier(random_state=42), xgb.XGBClassifier(random_state=42)]
    param_grids = [param_grid_lr, param_grid_nb, param_grid_svc, param_grid_knn, param_grid_dt,
                    param_grid_rf, param_grid_gbm, param_grid_lgb, param_grid_xgb]

    print(f'{target} prediction\n')
    print(f'---------------------------------------------------------\n')
    if target != 'ESG_Rating_band_by_E/S/G_Rating_band':
        for i in range(len(model_names)):
            if model_names[i] == 'XGBoost':
                y_encoded = label_encoder.transform(y)
                X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, shuffle=True, stratify=y_encoded)
            else:
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

            param_grid = param_grids[i]
            custom_scorer = make_scorer(f1_score, greater_is_better=True, average="weighted")
            column_transformer = ColumnTransformer([('scaling', StandardScaler(), X_train.columns) ], remainder='passthrough')

            pipeline = Pipeline(steps=[('scaling', column_transformer), ('classifier', names_in_python[i])])

            model = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=custom_scorer,
                                n_jobs=-1, cv=5).fit(X_train, y_train)

            y_pred_train = model.predict(X_train)
            y_pred_probs_train = model.predict_proba(X_train)

            y_pred_test = model.predict(X_test)
            y_pred_probs_test = model.predict_proba(X_test)

            print(f'{model_names[i]}\n')
            fig, ax = plt.subplots(1, 2, figsize=(10, 4), dpi=120)

            cm_train = confusion_matrix(y_train, y_pred_train)
            cmp_train = ConfusionMatrixDisplay(cm_train, display_labels=model.classes_)

            cm_test = confusion_matrix(y_test, y_pred_test)
            cmp_test = ConfusionMatrixDisplay(cm_test, display_labels=model.classes_)

            cmp_train.plot(ax=ax[0])
            ax[0].set_title('Confusion Matrix (Train)')

            cmp_test.plot(ax=ax[1])
            ax[1].set_title('Confusion Matrix (Test)')

            plt.tight_layout()
            plt.savefig(f'/content/ESG/plots/{target}_model_{model_names[i]}.png')
            plt.show()

            print('Train:\n', classification_report(y_train, y_pred_train))
            print('Test:\n', classification_report(y_test, y_pred_test))

            result = pd.DataFrame(
                [
                    [
                        model_names[i],
                        'Train',
                        round(roc_auc_score(y_train, y_pred_probs_train, multi_class="ovr", average="weighted"), 5),
                        round(accuracy_score(y_train, y_pred_train), 5),
                        round(precision_score(y_train, y_pred_train, average="weighted"), 5),
                        round(recall_score(y_train, y_pred_train, average="weighted"), 5),
                        round(f1_score(y_train, y_pred_train, average="weighted"), 5)
                    ],
                    [
                        model_names[i],
                        'Test',
                        round(roc_auc_score(y_test, y_pred_probs_test, multi_class="ovr", average="weighted"), 5),
                        round(accuracy_score(y_test, y_pred_test), 5),
                        round(precision_score(y_test, y_pred_test, average="weighted"), 5),
                        round(recall_score(y_test, y_pred_test, average="weighted"), 5),
                        round(f1_score(y_test, y_pred_test, average="weighted"), 5)
                    ]
                ],
                columns=['Model', 'Prediction Sample', 'ROC-AUC', 'Accuracy', 'Precision', 'Recall', 'F1']
            )
            result.to_excel(f'/content/ESG/results/{target}_model_{model_names[i]}.xlsx', index=False)
            print(f'---------------------------------------------------------\n')
            print(f'---------------------------------------------------------\n')

        results = pd.DataFrame()
        for model in model_names:
            res = pd.read_excel(f'/content/ESG/results/{target}_model_{model}.xlsx')
            results = pd.concat([results, res])

        results = results.reset_index(drop=True)
        results.to_excel(f'/content/ESG/results/{target}_results.xlsx', index=False)
        display(results)

        df = results[results['Prediction Sample'] == 'Test']
        best_metric_index = df['F1'].idxmax()
        best_model_name = df.loc[best_metric_index]['Model']

        best_model_index = model_names.index(best_model_name)
        param_grid = param_grids[best_model_index]
        custom_scorer = make_scorer(f1_score, greater_is_better=True, average="weighted")
        column_transformer = ColumnTransformer([('scaling', StandardScaler(), X_train.columns)], remainder='passthrough')

        pipeline = Pipeline(steps=[('scaling', column_transformer), ('classifier', names_in_python[best_model_index])])

        if best_model_name == 'XGBoost':
            y_encoded = label_encoder.transform(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, shuffle=True, stratify=y_encoded)
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

        model = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=custom_scorer,
                            n_jobs=-1, cv=5).fit(X_train, y_train)

        y_pred_test = model.predict(X_test)
        if np.issubdtype(y_pred_test.dtype, np.number):
                    y_pred_test = label_encoder.inverse_transform(y_pred_test)
        y_pred_probs_test = model.predict_proba(X_test)

        final_res = features_data.loc[y_test.index][['company', target]]
        final_res[f'Prediction({target})'] = y_pred_test
        final_res[f'Probability({target})=A'] = y_pred_probs_test[:, 0]
        final_res[f'Probability({target})=B'] = y_pred_probs_test[:, 1]
        final_res[f'Probability({target})=C'] = y_pred_probs_test[:, 2]
        final_res = final_res.sort_values(by='company').reset_index(drop=True)
        display(final_res)

        print(f'Best model is {best_model_name}\n')
        print(f'---------------------------------------------------------\n')

        best_model_result = pd.DataFrame([[best_model_name, 'Test',
                      round(roc_auc_score(y_test, y_pred_probs_test, multi_class="ovr", average="weighted"), 5),
                      round(accuracy_score(y_test, y_pred_test), 5),
                      round(precision_score(y_test, y_pred_test, average="weighted"), 5),
                      round(recall_score(y_test, y_pred_test, average="weighted"), 5),
                      round(f1_score(y_test, y_pred_test, average="weighted"), 5)]],
                      columns=['Model', 'Prediction Sample', 'ROC-AUC', 'Accuracy', 'Precision', 'Recall', 'F1'])
        display(best_model_result)
        final_res.to_excel(f'/content/ESG/results/{target}_final_result.xlsx', index=False)
    else:
        targets = ['ESG_Rating_band', 'E_Rating_band', 'S_Rating_band', 'G_Rating_band']

        X = features_data.drop(columns=['ESG_Rank', 'ESG_Rating_band', 'E_Rank', 'E_Rating_band', 'S_Rank', 'S_Rating_band', 'G_Rank', 'G_Rating_band', 'company'])
        for i, model in enumerate(best_models):
            y = features_data[targets[i]]
            best_model_index = model_names.index(model)
            param_grid = param_grids[best_model_index]
            custom_scorer = make_scorer(f1_score, greater_is_better=True, average="weighted")

            if best_models[i] == 'XGBoost':
                label_encoder = LabelEncoder().fit(y)
                y_encoded = label_encoder.transform(y)
                if features_data_test is not None:
                    X_test = features_data_test.drop(columns=['company'])
                    X_train, y_train = X, y_encoded
                else:
                    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, shuffle=True, stratify=y_encoded)
            else:
                if features_data_test is not None:
                    X_test = features_data_test.drop(columns=['company'])
                    X_train, y_train = X, y
                else:
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

            column_transformer = ColumnTransformer([('scaling', StandardScaler(), X_train.columns)], remainder='passthrough')

            pipeline = Pipeline(steps=[('scaling', column_transformer), ('classifier', names_in_python[best_model_index])])

            model = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=custom_scorer,
                                n_jobs=-1, cv=5).fit(X_train, y_train)

            if i == 0:
                y_pred_test_esg = model.predict(X_test)
                if np.issubdtype(y_pred_test_esg.dtype, np.number):
                    y_pred_test_esg = label_encoder.inverse_transform(y_pred_test_esg)
                y_pred_probs_test_esg = model.predict_proba(X_test)
            elif i == 1:
                y_pred_test_e = model.predict(X_test)
                if np.issubdtype(y_pred_test_e.dtype, np.number):
                    y_pred_test_e = label_encoder.inverse_transform(y_pred_test_e)
                y_pred_probs_test_e = model.predict_proba(X_test)
            elif i == 2:
                y_pred_test_s = model.predict(X_test)
                if np.issubdtype(y_pred_test_s.dtype, np.number):
                    y_pred_test_s = label_encoder.inverse_transform(y_pred_test_s)
                y_pred_probs_test_s = model.predict_proba(X_test)
            elif i == 3:
                y_pred_test_g = model.predict(X_test)
                if np.issubdtype(y_pred_test_g.dtype, np.number):
                    y_pred_test_g = label_encoder.inverse_transform(y_pred_test_g)
                y_pred_probs_test_g = model.predict_proba(X_test)

        rating_brand_encoding = {'A': 3, 'B': 2, 'C': 1}

        if features_data_test is not None:
            final_res = features_data_test[['company']]
        else:
            final_res = features_data.loc[y_test.index][['company', 'ESG_Rating_band']]

        final_res['Prediction(ESG_Rating_band)'] = y_pred_test_esg
        final_res['Prediction(E_Rating_band)'] = y_pred_test_e
        final_res['Prediction(S_Rating_band)'] = y_pred_test_s
        final_res['Prediction(G_Rating_band)'] = y_pred_test_g

        final_res['Prediction(E_Rating_band)_encoded'] = final_res['Prediction(E_Rating_band)'].map(rating_brand_encoding)
        final_res['Prediction(S_Rating_band)_encoded'] = final_res['Prediction(S_Rating_band)'].map(rating_brand_encoding)
        final_res['Prediction(G_Rating_band)_encoded'] = final_res['Prediction(G_Rating_band)'].map(rating_brand_encoding)

        final_res['Mean(Prediction(E/S/G_Rating_band))'] = ((final_res['Prediction(E_Rating_band)_encoded'] + final_res['Prediction(S_Rating_band)_encoded'] + final_res['Prediction(G_Rating_band)_encoded']) / 3).round()
        final_res['Mean(Prediction(E/S/G_Rating_band))'] = final_res['Mean(Prediction(E/S/G_Rating_band))'].map({value: key for key, value in rating_brand_encoding.items()})

        final_res.drop(columns=['Prediction(E_Rating_band)_encoded', 'Prediction(S_Rating_band)_encoded', 'Prediction(G_Rating_band)_encoded'], inplace=True)

        final_res = final_res.sort_values(by='company').reset_index(drop=True)
        final_res.to_excel('/content/ESG/results/final_result.xlsx', index=False)
        display(final_res)

        if features_data_test is None:
            final_res_scores = pd.DataFrame([
                [
                    'Prediction(ESG_Rating_band)',
                    round(accuracy_score(final_res['ESG_Rating_band'], final_res['Prediction(ESG_Rating_band)']), 5),
                    round(precision_score(final_res['ESG_Rating_band'], final_res['Prediction(ESG_Rating_band)'], average="weighted"), 5),
                    round(recall_score(final_res['ESG_Rating_band'], final_res['Prediction(ESG_Rating_band)'], average="weighted"), 5),
                    round(f1_score(final_res['ESG_Rating_band'], final_res['Prediction(ESG_Rating_band)'], average="weighted"), 5)
                ],
                [
                    'Mean(Prediction(E/S/G_Rating_band))',
                    round(accuracy_score(final_res['ESG_Rating_band'], final_res['Mean(Prediction(E/S/G_Rating_band))']), 5),
                    round(precision_score(final_res['ESG_Rating_band'], final_res['Mean(Prediction(E/S/G_Rating_band))'], average="weighted"), 5),
                    round(recall_score(final_res['ESG_Rating_band'], final_res['Mean(Prediction(E/S/G_Rating_band))'], average="weighted"), 5),
                    round(f1_score(final_res['ESG_Rating_band'], final_res['Mean(Prediction(E/S/G_Rating_band))'], average="weighted"), 5)
                ]],
                columns=['Experiment', 'Accuracy', 'Precision', 'Recall', 'F1']
            )
            final_res_scores.to_excel('/content/ESG/results/final_res_scores.xlsx', index=False)
            display(final_res_scores)
        return final_res

In [ ]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    "accept": "application/xml"
}

timeout = (5, 25)

In [ ]:
folder_name = 'PDFs'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Testing model on new data to predict rating

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1uUgKmlYBCWxFTMvbcwKQDCblTQhKOsZ_zPs0VCRQBEE/export?format=xlsx'
response = requests.get(url)

excel = pd.ExcelFile(BytesIO(response.content))
sheets = excel.sheet_names
sheets

['Методика',
 'Компании СЗ+RAEX',
 'Copy of Компании СЗ',
 'Общий отраслевой рейтинг',
 'Региональный отраслевой рейтинг',
 'Общий глобальный рейтинг ',
 'Региональный глобальный рейтинг',
 'База данных',
 'Технический лист',
 'Copy of Ранги и места',
 'Компании RAEX',
 'MOEX']

In [ ]:
data = pd.read_excel(excel, sheet_name='Компании СЗ+RAEX')
data.drop(['Unnamed: 0', 'Unnamed: 17', 'Unnamed: 18'], axis=1, inplace=True)
data.columns = ['Отрасль', 'Бренд и Юрлицо', 'Веб-сайт', 'ИНН', 'МOEX', 'Регион', 'Устойчивое развитие, Социальная отчетность, ESG', 'Экология', 'Забота о потребителях, защита прав потребителей',
                'Забота о работниках, социальные программы', 'Поставщикам, Закупки', 'Корпоративное управление, совет директоров, высший менеджмент',
                'Миссия, ценности', 'Этический кодекс', 'Антикоррупция', 'Раскрытие информации']
data = data.drop([0, 1], axis=0).reset_index(drop=True)
data['Отрасль'] = data['Отрасль'].fillna(method='ffill')
data = data[data['Регион'] == 'СЗФО']
data

,Отрасль,Бренд и Юрлицо,Веб-сайт,ИНН,МOEX,Регион,"Устойчивое развитие, Социальная отчетность, ESG",Экология,"Забота о потребителях, защита прав потребителей","Забота о работниках, социальные программы","Поставщикам, Закупки","Корпоративное управление, совет директоров, высший менеджмент","Миссия, ценности",Этический кодекс,Антикоррупция,Раскрытие информации
18,Тяжелая промышленность,«Группа ПОЛИПЛАСТИК»,https://www.polyplastic.ru,5021013384,NaN,RAEX,нет,https://www.polyplastic.ru/company/eco,нет,нет,нет,нет,нет,нет,https://www.polyplastic.ru/upload/company/anti...,https://www.polyplastic.ru/documents
19,Тяжелая промышленность,«Упаковочные системы»,https://www.packaging-systems.ru,7706017070,NaN,RAEX,https://www.packaging-systems.ru/sustainable/,https://www.packaging-systems.ru/sustainable/,нет,нет,нет,нет,https://www.packaging-systems.ru/#missions,нет,нет,нет
21,ХИМИЧЕСКАЯ ПРОМ,«ФосАгро»,https://www.phosagro.ru,7736216869,*,RAEX,https://www.phosagro.ru/sustainability/,https://www.phosagro.ru/sustainability/ecology/,https://www.phosagro.ru/production/attention/,https://www.phosagro.ru/sustainability/social-...,https://www.phosagro.ru/procurement/,https://www.phosagro.ru/about/management/,https://www.phosagro.ru/sustainability/,https://ar2022.phosagro.ru/pdf/ar/ru/corporate...,нет,https://www.phosagro.ru/ori/phosagro/company_c...
22,ХИМИЧЕСКАЯ ПРОМ,«Уралкалий»,https://www.uralkali.com/ru/,5911029807,NaN,RAEX,https://www.uralkali.com/ru/sustainability/,https://www.uralkali.com/ru/sustainability/env...,https://www.uralkali.com/ru/buyers/quality/,https://www.uralkali.com/ru/sustainability/our...,https://www.uralkali.com/ru/tenders/obrashchen...,https://www.uralkali.com/ru/about/corporate_go...,https://www.uralkali.com/ru/about/mission/,https://www.uralkali.com/ru/sustainability/eth...,https://www.uralkali.com/ru/sustainability/eth...,https://www.uralkali.com/ru/investors/reportin...
23,ХИМИЧЕСКАЯ ПРОМ,"«Уралхим», ОХК",https://www.uralchem.ru,7703647595,NaN,RAEX,https://www.uralchem.ru/esg/,https://www.uralchem.ru/esg/ekologicheskiy-asp...,https://www.uralchem.ru/products/vnimaniyu-pok...,https://www.uralchem.ru/esg/sotsialnyy-aspekt-s/,https://www.uralchem.ru/purchase/kak-stat-post...,https://www.uralchem.ru/corporate_management/,https://www.uralchem.ru/about/mission/,нет,https://www.uralchem.ru/corporate_management/a...,https://www.uralchem.ru/esg/esg-dokumenty/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,СЕЛЬХОЗ,«Группа Черкизово»,https://cherkizovo-group.com/,7718560636,NaN,RAEX,https://cherkizovo-group.com/sustainable-devel...,Нет,Нет,https://cherkizovo-group.com/career/,Нет,нет,https://cherkizovo-group.com/about/,нет,нет,https://cherkizovo-group.com/investors/
304,СЕЛЬХОЗ,«Эфирное»,https://www.efko.ru,3122000300,NaN,RAEX,нет,Нет,Нет,https://www.efko.ru/career/,Нет,нет,https://www.efko.ru/about/#mission,нет,нет,Нет
305,СЕЛЬХОЗ,«Торговая компания «Мираторг»,http://www.miratorg.ru/,5009072150,NaN,RAEX,https://miratorg.ru/about/sustainable_developm...,Нет,Нет,https://miratorg.ru/career/vacancies/,Нет,https://miratorg.ru/about/directors/,нет,нет,нет,https://e-disclosure.ru/portal/company.aspx?id...
306,ВОДОСНАБЖЕНИЕ,РОСВОДОКАНАЛ,http://www.rosvodokanal.ru/,5027116065,NaN,RAEX,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/ustoychivoe-razvit...,https://www.rosvodokanal.ru/about/zakupki/,https://www.rosvodokanal.ru/about/rukovodstvo/,https://www.rosvodokanal.ru/about/missiya/,нет,нет,https://www.rosvodokanal.ru/ustoychivoe-razvit...


In [ ]:
df = data.melt(id_vars=['Отрасль', 'Бренд и Юрлицо', 'Веб-сайт', 'ИНН'], value_vars=['Устойчивое развитие, Социальная отчетность, ESG', 'Экология', 'Забота о потребителях, защита прав потребителей',
                'Забота о работниках, социальные программы', 'Поставщикам, Закупки', 'Корпоративное управление, совет директоров, высший менеджмент',
                'Миссия, ценности', 'Этический кодекс', 'Антикоррупция', 'Раскрытие информации'], var_name='url_type', value_name='url')
df = df.drop(columns=['url_type'])
df

,Отрасль,Бренд и Юрлицо,Веб-сайт,ИНН,url
0,Тяжелая промышленность,«Группа ПОЛИПЛАСТИК»,https://www.polyplastic.ru,5021013384,нет
1,Тяжелая промышленность,«Упаковочные системы»,https://www.packaging-systems.ru,7706017070,https://www.packaging-systems.ru/sustainable/
2,ХИМИЧЕСКАЯ ПРОМ,«ФосАгро»,https://www.phosagro.ru,7736216869,https://www.phosagro.ru/sustainability/
3,ХИМИЧЕСКАЯ ПРОМ,«Уралкалий»,https://www.uralkali.com/ru/,5911029807,https://www.uralkali.com/ru/sustainability/
4,ХИМИЧЕСКАЯ ПРОМ,"«Уралхим», ОХК",https://www.uralchem.ru,7703647595,https://www.uralchem.ru/esg/
...,...,...,...,...,...
1405,СЕЛЬХОЗ,«Группа Черкизово»,https://cherkizovo-group.com/,7718560636,https://cherkizovo-group.com/investors/
1406,СЕЛЬХОЗ,«Эфирное»,https://www.efko.ru,3122000300,Нет
1407,СЕЛЬХОЗ,«Торговая компания «Мираторг»,http://www.miratorg.ru/,5009072150,https://e-disclosure.ru/portal/company.aspx?id...
1408,ВОДОСНАБЖЕНИЕ,РОСВОДОКАНАЛ,http://www.rosvodokanal.ru/,5027116065,https://www.rosvodokanal.ru/ustoychivoe-razvit...


In [ ]:
words_to_filter = ['Нет', 'нет', 'не работает']
pattern = '|'.join(words_to_filter)

df['url'] = df['url'].apply(lambda x: np.nan if pd.notna(x) and pd.Series(x).str.contains(pattern).any() else x)
df = df.dropna(subset=["url"])
df['url'] = df['url'].apply(clear_string)
df = df.explode('url')
df = df.sort_values(by='Бренд и Юрлицо').reset_index(drop=True)
df

,Отрасль,Бренд и Юрлицо,Веб-сайт,ИНН,url
0,ХИМИЧЕСКАЯ ПРОМ,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/,3448003962,https://www.kaustik.ru/ru/index.php/o-kompanii...
1,ХИМИЧЕСКАЯ ПРОМ,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/,3448003962,https://www.kaustik.ru/ru/index.php/o-kompanii...
2,ХИМИЧЕСКАЯ ПРОМ,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/,3448003962,sustainability-report-kaustik.pdf
3,ХИМИЧЕСКАЯ ПРОМ,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/,3448003962,https://www.kaustik.ru/ru/index.php/partneram/...
4,ХИМИЧЕСКАЯ ПРОМ,"""Каустик"" (г. Волгоград)",https://www.kaustik.ru/ru/,3448003962,https://www.kaustik.ru/ru/index.php/o-kompanii...
...,...,...,...,...,...
766,НЕФТЕГАЗОВАЯ ПРОМ,Якутская топливно-энергетическая компания (ЯТЭК),http://www.yatec.ru/,1435027335,https://yatec.ru/razvitie/ecology/
767,НЕФТЕГАЗОВАЯ ПРОМ,Якутская топливно-энергетическая компания (ЯТЭК),http://www.yatec.ru/,1435027335,https://www.yatec.ru/aktsioneram/raskrytie-inf...
768,НЕФТЕГАЗОВАЯ ПРОМ,Якутская топливно-энергетическая компания (ЯТЭК),http://www.yatec.ru/,1435027335,https://yatec.ru/razvitie/safety/
769,НЕФТЕГАЗОВАЯ ПРОМ,Якутская топливно-энергетическая компания (ЯТЭК),http://www.yatec.ru/,1435027335,https://yatec.ru/razvitie/ecology/


In [ ]:
urls = df.set_index('url')['Бренд и Юрлицо'].to_dict()
len(urls)

701

In [ ]:
%%time
chunk_size = 5
links = pd.DataFrame()
i = 0
for URLs_chunk in chunkify(urls, chunk_size):
    chunk_results, i = process_chunk(URLs_chunk, i, get_all_links)
    try:
        links = pd.concat([links, chunk_results])
        del chunk_results
        gc.collect()
    except Exception as err:
        continue

print("Links are collected!")

links = links.drop_duplicates().reset_index(drop=True)

words_to_filter = ['.xlsx', '.docx', '.7z', '.rar', '.JPG', '.xlsb', '.xls', '.zip', '.mp4', '.jpg', '.png', '.doc', '.PNG', 'download_file.php']
links = links[~links['urls'].str.contains('|'.join(words_to_filter))].reset_index(drop=True)
links.to_csv('links.csv', encoding='utf-8')

DONE 0 "Бронка Групп"
DONE 1 "Балтийская жемчужина"
DONE 2 "Балтийская жемчужина"
DONE 3 Бронка Групп
DONE 4 "ВкусВилл"
DONE 5 "ВкусВилл"
DONE 6 "ВкусВилл"
DONE 7 "ВкусВилл"
DONE 8 "ВкусВилл"
DONE 9 "ВкусВилл"
DONE 10 "ВкусВилл"
ERROR 11 "Открытие" Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome-headless-shell=125.0.6422.76)
Stacktrace:
#0 0x55e3b32bfe9a <unknown>
#1 0x55e3b2fa945c <unknown>
#2 0x55e3b2fa0fd0 <unknown>
#3 0x55e3b2f90b32 <unknown>
#4 0x55e3b2f92162 <unknown>
#5 0x55e3b2f90ded <unknown>
#6 0x55e3b2f90503 <unknown>
#7 0x55e3b2f903f5 <unknown>
#8 0x55e3b2f8e336 <unknown>
#9 0x55e3b2f8e83a <unknown>
#10 0x55e3b2fabea7 <unknown>
#11 0x55e3b3037ed5 <unknown>
#12 0x55e3b30184b2 <unknown>
#13 0x55e3b30372cc <unknown>
#14 0x55e3b3018253 <unknown>
#15 0x55e3b2fe81c7 <unknown>
#16 0x55e3b2fe8b3e <unknown>
#17 0x55e3b32862db <unknown>
#18 0x55e3b328a387 <unknown>
#19 0x55e3b3272e0e <unknown>
#20 0x55e3b328ae52 <unknown>
#21 0x55e3b32577af <unknown>
#22 0x

In [ ]:
folder_path = 'PDFs'
output_path = 'PDFs.zip'

zip_folder(folder_path, output_path)

In [ ]:
links = pd.read_csv('links.csv', index_col=0)

with zipfile.ZipFile('PDFs.zip', 'r') as zipf:
    zipf.extractall('PDFs')

len(links.company.unique())

urls = links.set_index('urls')['company'].to_dict()
len(urls)

5342

In [ ]:
%%time
chunk_size = 4
parsed_data = pd.DataFrame()
i = 0
for URLs_chunk in chunkify(urls, chunk_size):
    chunk_results, i = process_chunk(URLs_chunk, i, parsing)
    try:
        parsed_data = pd.concat([parsed_data, chunk_results])
        del chunk_results
        gc.collect()
    except Exception as err:
        continue

print("Parcing is finished!")

parsed_data = parsed_data.drop_duplicates()
parsed_data = parsed_data.reset_index(drop=True)
parsed_data.to_csv('parsed_data_test.csv', encoding='utf-8')

Выходные данные были обрезаны до нескольких последних строк (5000).
DONE 892 «Петрополь»
DONE 893 «Петрополь»
DONE 894 «Петрополь»
DONE 895 «Лузалес»,
DONE 896 АВТОТОР (Калининград)
DONE 897 АВТОТОР (Калининград)
DONE 898 АВТОТОР (Калининград)
DONE 899 «Лузалес»,
DONE 900 АВТОТОР (Калининград)
DONE 901 АВТОТОР (Калининград)
DONE 902 АВТОТОР (Калининград)
DONE 903 АВТОТОР (Калининград)
DONE 904 АВТОТОР (Калининград)
DONE 905 АВТОТОР (Калининград)
DONE 906 АВТОТОР (Калининград)
DONE 907 АВТОТОР (Калининград)
DONE 908 АВТОТОР (Калининград)
DONE 909 АВТОТОР (Калининград)
DONE 910 АВТОТОР (Калининград)
DONE 911 АВТОТОР (Калининград)
DONE 912 АВТОТОР (Калининград)
DONE 913 АВТОТОР (Калининград)
DONE 914 АВТОТОР (Калининград)
DONE 915 АВТОТОР (Калининград)
DONE 916 АВТОТОР (Калининград)
DONE 917 АВТОТОР (Калининград)
DONE 918 АВТОТОР (Калининград)
DONE 919 АВТОТОР (Калининград)
DONE 920 АВТОТОР (Калининград)
DONE 921 АВТОТОР (Калининград)
DONE 922 АВТОТОР (Калининград)
DONE 923 АВТОТОР (Калин

In [ ]:
parsed_data

,company,url,text
0,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,социальный ответственность компанииназадый нас...
1,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,поддержка исторический культурный достояние кр...
2,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,санктпетербург
3,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,ул красный текстильщик далее
4,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,главный
...,...,...,...
565455,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,дистрибьюторский сеть estel охватывать регион ...
565456,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,найти
565457,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,подписаться новость
565458,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,новость карта сайт контакт


In [ ]:
data = pd.read_csv('parsed_data_test.csv', index_col=0)
data['text_cyr'] = data['text'].str.replace(r"[^а-яА-Я\s]+", '', regex=True).str.strip()
data['text_cyr_words'] = data['text_cyr'].str.split()
data = data.replace(r'^s*$', float('NaN'), regex=True).dropna().reset_index(drop=True)
data

,company,url,text,text_cyr,text_cyr_words
0,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,социальный ответственность компанииназадый нас...,социальный ответственность компанииназадый нас...,"[социальный, ответственность, компанииназадый,..."
1,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,поддержка исторический культурный достояние кр...,поддержка исторический культурный достояние кр...,"[поддержка, исторический, культурный, достояни..."
2,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,санктпетербург,санктпетербург,[санктпетербург]
3,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,ул красный текстильщик далее,ул красный текстильщик далее,"[ул, красный, текстильщик, далее]"
4,Бронка Групп,https://www.bronkagroup.ru/about/sotsialnaya-o...,главный,главный,[главный]
...,...,...,...,...,...
526381,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,дистрибьюторский сеть estel охватывать регион ...,дистрибьюторский сеть охватывать регион росси...,"[дистрибьюторский, сеть, охватывать, регион, р..."
526382,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,найти,найти,[найти]
526383,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,подписаться новость,подписаться новость,"[подписаться, новость]"
526384,“Юникосметик” (бренд Estel),https://estel.pro/produkciya/uhod-za-kozhej,новость карта сайт контакт,новость карта сайт контакт,"[новость, карта, сайт, контакт]"


In [ ]:
%%time
data_company = data.groupby(['company'])[['text_cyr_words']].apply(sum).reset_index().sort_values(by=['company'])
data_company['text_cyr_words'] = data_company['text_cyr_words'].agg(pd.unique)
data_company.to_csv('parsed_data_company_test.csv', encoding='utf-8')

CPU times: user 8min 51s, sys: 3.8 s, total: 8min 55s
Wall time: 9min 10s


In [ ]:
data_company = pd.read_csv('parsed_data_company_test.csv', index_col=0)
data_company['text_cyr_words'] = data_company['text_cyr_words'].apply(lambda x: literal_eval(x.replace(' ', ','))).tolist()
data_company

,company,text_cyr_words
0,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба..."
1,"""Балтийская жемчужина""","[способ, покупка, недвижимость, жк, балтийский..."
2,"""ВкусВилл""","[экология, поддержка, вкусвилл, выбрать, спосо..."
3,"""Открытие""","[офис, банкомат, банк, открытиечастный, клиент..."
4,"""РЕСО-Гарантия""","[команда, внимание, ресогарантия, урегулироват..."
...,...,...
122,“Активный компонент”,"[компания, ао, активный, компонент, заказать, ..."
123,“Воздушные ворота Северной столицы”,"[авиационный, учебный, центр, вылетать, добрат..."
124,“Ленсвет”,"[структура, руководство, маршрут, прогулка, ве..."
125,“Эко Лэнд”,"[эко, лэнд, вывоз, сортировка, тбо, главный, к..."


In [ ]:
topics_data = pd.read_csv('topics_data.csv', index_col=0)
topics_data['topics_words'] = topics_data['topics_words'].apply(lambda x: literal_eval(x.replace(', ', ','))).tolist()
topics_companies_data = data_company.merge(topics_data, how='cross')
topics_companies_data

,company,text_cyr_words,topic_name,topic_num,topics_words
0,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии..."
1,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ..."
2,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в..."
3,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Биоразнообразие,3,"[москвемосква, регионахсибирьо, улбурмакинскии..."
4,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Биоразнообразие,4,"[развитииновость, развитияпочесть, развитиенов..."
...,...,...,...,...,...
7615,“Юникосметик” (бренд Estel),"[завод, поиск, сайт, найти, салон, россия, выб...",Отходы,55,"[ремонтник, ремонтвоити, ремонта, ремонтускидк..."
7616,“Юникосметик” (бренд Estel),"[завод, поиск, сайт, найти, салон, россия, выб...",Безопасность продукта,56,"[годфинансовыи, годафинансовыи, годуфинансовыи..."
7617,“Юникосметик” (бренд Estel),"[завод, поиск, сайт, найти, салон, россия, выб...",Обучение и развитие,57,"[обучениязабота, обучениедля, обучениявысокии,..."
7618,“Юникосметик” (бренд Estel),"[завод, поиск, сайт, найти, салон, россия, выб...",Лидерство,58,"[авиаперевозчик, авиазаправочныи, авиаперевозк..."


In [ ]:
%%time
cosine_similarities = []
for index, row in topics_companies_data.iterrows():
    tf_idf = TfidfVectorizer()

    text_cyr_words = np.array([elm for elm in row['text_cyr_words'] if isinstance(elm, str)])
    topics_words = np.array([elm for elm in row['topics_words'] if isinstance(elm, str)])

    text_cyr_words = [' '.join(text_cyr_words)]
    topics_words = [' '.join(topics_words)]

    data_company_tf = tf_idf.fit_transform(text_cyr_words)
    topics_data_tf = tf_idf.transform(topics_words)

    cosine_similarity = linear_kernel(data_company_tf, topics_data_tf).flatten()
    cosine_similarities.append(cosine_similarity[0])

CPU times: user 1min 43s, sys: 397 ms, total: 1min 44s
Wall time: 1min 45s


In [ ]:
topics_companies_data['cosine_similarity'] = cosine_similarities
topics_companies_data.head()

,company,text_cyr_words,topic_name,topic_num,topics_words,cosine_similarity
0,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,0,"[раичихинскии, улмихалковскии, юлмалоохтинскии...",0.000000
1,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,1,"[банкекредитныи, кредитабанк, сахакредитбанк, ...",0.023202
2,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Безопасность продукта,2,"[пон, гыд, ыд, тмм, ич, ек, жижа, атво, тпу, в...",0.018944
3,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Биоразнообразие,3,"[москвемосква, регионахсибирьо, улбурмакинскии...",0.000000
4,"""Альфа-Банк""","[структура, корпоративный, управление, альфаба...",Биоразнообразие,4,"[развитииновость, развитияпочесть, развитиенов...",0.000000


In [ ]:
features_data = (
    topics_companies_data
    .pivot(index=['company'], columns='topic_num', values='cosine_similarity')
    .reset_index()
    .add_prefix('topic_num_')
    .rename(columns={'topic_num_company': 'company'})
)

features_data.head()

topic_num,company,topic_num_0,topic_num_1,topic_num_2,topic_num_3,topic_num_4,topic_num_5,topic_num_6,topic_num_7,topic_num_8,...,topic_num_50,topic_num_51,topic_num_52,topic_num_53,topic_num_54,topic_num_55,topic_num_56,topic_num_57,topic_num_58,topic_num_59
0,"""Альфа-Банк""",0.0,0.023202,0.018944,0.000000,0.0,0.0,0.013395,0.029953,0.026791,...,0.000000,0.018944,0.018944,0.040186,0.023202,0.018944,0.023202,0.023202,0.018944,0.000000
1,"""Балтийская жемчужина""",0.0,0.000000,0.018137,0.000000,0.0,0.0,0.036274,0.025649,0.025649,...,0.031414,0.031414,0.018137,0.000000,0.000000,0.018137,0.018137,0.031414,0.025649,0.000000
2,"""ВкусВилл""",0.0,0.000000,0.017555,0.000000,0.0,0.0,0.021500,0.012413,0.017555,...,0.032842,0.027756,0.021500,0.000000,0.000000,0.012413,0.000000,0.027756,0.017555,0.012413
3,"""Открытие""",0.0,0.000000,0.041867,0.000000,0.0,0.0,0.000000,0.029604,0.029604,...,0.051276,0.029604,0.000000,0.000000,0.000000,0.000000,0.029604,0.029604,0.000000,0.029604
4,"""РЕСО-Гарантия""",0.0,0.000000,0.000000,0.027918,0.0,0.0,0.034193,0.027918,0.027918,...,0.034193,0.039482,0.034193,0.000000,0.019741,0.034193,0.000000,0.027918,0.039482,0.000000


In [ ]:
features_data.to_csv('features_data_test.csv', encoding='utf-8')

In [ ]:
features_data = pd.read_csv('features_data_train.csv', index_col=0)

features_data['ESG_Rank'] = np.where(features_data['ESG_Rank'] == '144-145', '144', features_data['ESG_Rank'])
features_data['ESG_Rank'] = np.where(features_data['ESG_Rank'] == '157-160', '157', features_data['ESG_Rank'])
features_data['E_Rank'] = np.where(features_data['E_Rank'] == '126-127', '126', features_data['E_Rank'])
features_data['E_Rank'] = np.where(features_data['E_Rank'] == '133-134', '133', features_data['E_Rank'])
features_data['E_Rank'] = np.where(features_data['E_Rank'] == '145-146', '145', features_data['E_Rank'])
features_data['E_Rank'] = np.where(features_data['E_Rank'] == '157-160', '157', features_data['E_Rank'])
features_data['S_Rank'] = np.where(features_data['S_Rank'] == '157-160', '157', features_data['S_Rank'])
features_data['G_Rank'] = np.where(features_data['G_Rank'] == '68-69', '68', features_data['G_Rank'])
features_data['G_Rank'] = np.where(features_data['G_Rank'] == '128-129', '128', features_data['G_Rank'])
features_data['G_Rank'] = np.where(features_data['G_Rank'] == '143-145', '143', features_data['G_Rank'])
features_data['G_Rank'] = np.where(features_data['G_Rank'] == '155-156', '155', features_data['G_Rank'])
features_data['G_Rank'] = np.where(features_data['G_Rank'] == '157-160', '157', features_data['G_Rank'])

features_data['ESG_Rank'] = features_data['ESG_Rank'].astype(int)
features_data['E_Rank'] = features_data['E_Rank'].astype(int)
features_data['S_Rank'] = features_data['S_Rank'].astype(int)
features_data['G_Rank'] = features_data['G_Rank'].astype(int)

features_data['ESG_Rating_band'] = np.where(features_data['ESG_Rating_band'] == 'M', 'CCC', features_data['ESG_Rating_band'])
features_data['ESG_Rating_band'] = np.where(features_data['ESG_Rating_band'].isin(['AAA', 'AA', 'A']), 'A', features_data['ESG_Rating_band'])
features_data['ESG_Rating_band'] = np.where(features_data['ESG_Rating_band'].isin(['BBB', 'BB', 'B']), 'B', features_data['ESG_Rating_band'])
features_data['ESG_Rating_band'] = np.where(features_data['ESG_Rating_band'].isin(['CCC', 'CC', 'C']), 'C', features_data['ESG_Rating_band'])

features_data['E_Rating_band'] = np.where(features_data['E_Rating_band'] == 'M', 'CCC', features_data['E_Rating_band'])
features_data['E_Rating_band'] = np.where(features_data['E_Rating_band'].isin(['AAA', 'AA']), 'A', features_data['E_Rating_band'])
features_data['E_Rating_band'] = np.where(features_data['E_Rating_band'].isin(['BBB', 'BB']), 'B', features_data['E_Rating_band'])
features_data['E_Rating_band'] = np.where(features_data['E_Rating_band'].isin(['CCC', 'CC']), 'C', features_data['E_Rating_band'])

features_data['S_Rating_band'] = np.where(features_data['S_Rating_band'] == 'M', 'CCC', features_data['S_Rating_band'])
features_data['S_Rating_band'] = np.where(features_data['S_Rating_band'].isin(['AAA', 'AA']), 'A', features_data['S_Rating_band'])
features_data['S_Rating_band'] = np.where(features_data['S_Rating_band'].isin(['BBB', 'BB']), 'B', features_data['S_Rating_band'])
features_data['S_Rating_band'] = np.where(features_data['S_Rating_band'].isin(['CCC', 'CC']), 'C', features_data['S_Rating_band'])

features_data['G_Rating_band'] = np.where(features_data['G_Rating_band'] == 'M', 'CCC', features_data['G_Rating_band'])
features_data['G_Rating_band'] = np.where(features_data['G_Rating_band'].isin(['AAA', 'AA']), 'A', features_data['G_Rating_band'])
features_data['G_Rating_band'] = np.where(features_data['G_Rating_band'].isin(['BBB', 'BB']), 'B', features_data['G_Rating_band'])
features_data['G_Rating_band'] = np.where(features_data['G_Rating_band'].isin(['CCC', 'CC']), 'C', features_data['G_Rating_band'])

features_data

In [ ]:
test_data = pd.read_csv('features_data_test.csv', index_col=0)
test_data.head()

In [ ]:
target = 'ESG_Rating_band_by_E/S/G_Rating_band'
best_models = ['LightGBM', 'LogisticRegression', 'LightGBM', 'LightGBM']

final_res = rating_band_prediction(features_data, target, best_models, test_data)

ESG_Rating_band_by_E/S/G_Rating_band prediction

---------------------------------------------------------

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000480 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1600
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 52
[LightGBM] [Info] Start training from score -2.389596
[LightGBM] [Info] Start training from score -1.203973
[LightGBM] [Info] Start training from score -0.497032
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Lig

,company,Prediction(ESG_Rating_band),Prediction(E_Rating_band),Prediction(S_Rating_band),Prediction(G_Rating_band),Mean(Prediction(E/S/G_Rating_band))
0,"""Альфа-Банк""",C,C,C,B,C
1,"""Балтийская жемчужина""",B,A,C,B,B
2,"""ВкусВилл""",C,C,C,A,B
3,"""Открытие""",C,C,C,C,C
4,"""РЕСО-Гарантия""",C,C,C,C,C
...,...,...,...,...,...,...
122,“Активный компонент”,C,C,C,C,C
123,“Воздушные ворота Северной столицы”,C,C,C,B,C
124,“Ленсвет”,C,C,C,C,C
125,“Эко Лэнд”,C,C,C,C,C


In [ ]:
final_res['Prediction(ESG_Rating_band)'].value_counts()

Prediction(ESG_Rating_band)
C    114
B     12
A      1
Name: count, dtype: int64

In [ ]:
folder_path = 'ESG'
output_path = 'ESG.zip'

zip_folder(folder_path, output_path)